In [1]:
import pandas as pd
import datetime
import numpy as np
import plotly.graph_objects as go

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv')
data.head()

,Fecha,Casos nuevos con sintomas,Casos totales,Casos recuperados,Fallecidos,Casos activos,Casos nuevos sin sintomas,Casos nuevos totales,Casos activos por FD,Casos activos por FIS,Casos recuperados por FIS,Casos recuperados por FD,Casos confirmados recuperados,Casos activos confirmados,Casos probables acumulados,Casos activos probables
0,2020-03-02,1.0,1.0,0.0,0.0,1.0,NaN,1.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2020-03-03,0.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2020-03-04,2.0,3.0,0.0,0.0,3.0,NaN,2.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,2020-03-05,1.0,4.0,0.0,0.0,4.0,NaN,1.0,4.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2020-03-06,1.0,5.0,0.0,0.0,5.0,NaN,1.0,5.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN


# Casos Nuevos

In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data["Fecha"], y=data["Casos nuevos totales"], mode='lines+markers'))
fig.update_layout(title_text='Casos nuevos COVID-19 Chile',  xaxis_title='Fecha')
fig.show()

# Camas Hospital

In [4]:
data_camas = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto24/CamasHospital_Diario_T.csv')
data_camas = data_camas.rename(columns={"Tipo de cama": "Fecha"})

fig = go.Figure()
for tipo in data_camas.columns[1:]:
    fig.add_trace(go.Scatter(x=data_camas["Fecha"], y=data_camas[tipo], mode='lines+markers', name=tipo))
fig.update_layout(title_text='Ocupación de camas hospitalarias',  xaxis_title='Fecha')
fig.show()

# Fallecidos DEIS

In [5]:
data_deis = pd.read_csv("https://raw.githubusercontent.com/joaquin-silva/covid-19-chile/master/datos_deis_covid.csv")
data_deis

,año,fecha,genero,edad,codigo_comuna,comuna,region,codigo_detalle,causa_detalle,codigo,causa
0,2020,2020-06-06,Hombre,80 a 84,1402,Camiña,De Tarapacá,U07.1,COVID-19 Confirmado,U07,COVID-19
1,2020,2020-06-18,Hombre,80 a 84,5101,Valparaíso,De Valparaíso,U07.1,COVID-19 Confirmado,U07,COVID-19
2,2020,2020-06-30,Hombre,75 a 79,13401,San Bernardo,Metropolitana de Santiago,U07.1,COVID-19 Confirmado,U07,COVID-19
3,2020,2020-04-25,Hombre,80 a 84,12101,Punta Arenas,De Magallanes y de La Antártica Chilena,U07.1,COVID-19 Confirmado,U07,COVID-19
4,2020,2020-07-01,Hombre,85 a 89,13131,San Ramón,Metropolitana de Santiago,U07.1,COVID-19 Confirmado,U07,COVID-19
...,...,...,...,...,...,...,...,...,...,...,...
10154,2020,2020-05-17,Mujer,65 a 69,13103,Cerro Navia,Metropolitana de Santiago,U07.1,COVID-19 Confirmado,U07,COVID-19
10155,2020,2020-06-15,Hombre,65 a 69,13110,La Florida,Metropolitana de Santiago,U07.1,COVID-19 Confirmado,U07,COVID-19
10156,2020,2020-06-28,Mujer,70 a 74,13111,La Granja,Metropolitana de Santiago,U07.1,COVID-19 Confirmado,U07,COVID-19
10157,2020,2020-06-01,Hombre,70 a 74,13110,La Florida,Metropolitana de Santiago,U07.1,COVID-19 Confirmado,U07,COVID-19


In [6]:
grouped = data_deis.groupby(["genero","edad"])
l_genero = []
l_edad = []
l_def = []
for name, group in grouped:
    l_genero.append(name[0])
    l_edad.append(name[1])
    l_def.append(group.shape[0])

if len(l_genero) < 42:
    l_genero.append("Hombre")
    l_edad.append("5 a 9")
    l_def.append(0)

data_deis_grouped = pd.DataFrame({"genero":l_genero,"edad":l_edad,"fallecidos":l_def})

sent_to_id = {"< 1":0,"1 a 4":1,"5 a 9":2,"10 a 14":3,"15 a 19":4,"20 a 24":5,"25 a 29":6,"30 a 34":7,"35 a 39":8,
              "40 a 44":9,"45 a 49":10,"50 a 54":11,"55 a 59":12,"60 a 64":13,"65 a 69":14,"70 a 74":15,"75 a 79":16,
              "80 a 84":17,"85 a 89":18,"90 a 99":19,"100 +":20}
data_deis_grouped["edad_id"] = data_deis_grouped["edad"].map(sent_to_id)
data_deis_grouped = data_deis_grouped.sort_values(by=["edad_id"])

fig = go.Figure()

for genero in ["Mujer","Hombre"]:
    data_aux = data_deis_grouped[data_deis_grouped["genero"]==genero]
    fig.add_trace(go.Bar(x=data_aux["edad"], y=data_aux["fallecidos"], name=genero))

fig.update_layout(barmode='group',title_text="Fallecidos por grupo etario COVID-19", xaxis_title="Edad", yaxis_title="Cantidad de fallecidos")
fig.show()